In [1]:
import pandas as pd
import nltk
import re
import unicodedata
from nltk import word_tokenize
from nltk import bigrams, trigrams, skipgrams
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

In [2]:
# Cargando el dataset
df = pd.read_csv('../data/train/train.csv')

In [3]:
df['Opinion']

0       "Las famosas momias de Guanajuato. El museo en...
1       "Allí puedes encontrar comida o souvenirs. Es ...
2       "Es tardado entrar al museo, una vez dentro no...
3       "Entiendo que a muchos les cautive o llamé la ...
4       "La fruta y verdura son poco frescos, al igual...
                              ...                        
4412    "This university is famous because of its beau...
4413    "Lo que no me gusto fue la comida, pero es muy...
4414    Esta excelente para pasar a sentarse unos minu...
4415    "No tuvimos la fortuna de entrar a verlo, ya q...
4416    "Excelente cuidado de las momias y en museos, ...
Name: Opinion, Length: 4417, dtype: object

## Limpieza  y tratamiento del texto de la Opinion

In [4]:
# Reemplazando NaN en la columna Opinion con una cadena vacía
df['Opinion'] = df['Opinion'].fillna('')

In [5]:
words = nltk.word_tokenize(' '.join(df['Opinion']))

In [6]:
tri_grams = list(trigrams(words))

print(tri_grams[:10])

[('``', 'Las', 'famosas'), ('Las', 'famosas', 'momias'), ('famosas', 'momias', 'de'), ('momias', 'de', 'Guanajuato'), ('de', 'Guanajuato', '.'), ('Guanajuato', '.', 'El'), ('.', 'El', 'museo'), ('El', 'museo', 'en'), ('museo', 'en', 'sí'), ('en', 'sí', 'es')]


In [7]:
def eliminar_acentos(texto):
    # Normalizamos el texto a la forma 'NFD' que separa los caracteres Unicode y sus diacríticos
    texto_normalizado = unicodedata.normalize('NFD', texto)
    # Filtramos todos los caracteres que no sean diacríticos (es decir, eliminamos los acentos)
    texto_sin_acentos = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
    return texto_sin_acentos

In [8]:
# Preprocesamiento de texto
def preprocess_text(text, remove_stopwords=False, lemmatize=False, include_punctuation=False, remove_special_chars_numbers=False, remove_other_x_chars=False):
    
    text = str(text).lower()

     # Eliminar caracteres especiales si remove_special_chars es True
    if remove_special_chars_numbers:
      texto_sin_acentos = eliminar_acentos(text)

      # Utilizamos re.sub() para dejar solo las letras y los espacios
      text = re.sub(r"[^a-zA-Z\s]", "", texto_sin_acentos)

    if remove_other_x_chars:
      text = re.sub(r'\b\w\b\s*', '', text)
    
    # Tokenización
    tokens = word_tokenize(text)

    # Eliminar signos de puntuación si include_punctuation es False
    if not include_punctuation:
        tokens = [word for word in tokens if word.isalnum()]

    # Eliminar stopwords
    if remove_stopwords:
        stop_words_english = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.isalnum() and word not in stop_words_english]

        stop_words_spanish = set(stopwords.words('spanish'))
        tokens = [word for word in tokens if word.isalnum() and word not in stop_words_spanish]
    
    # Lematización
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

### Generando gramas con valores enteros

In [9]:
def generate_gram(n, remove_stopwords=False, lemmatize=False,include_punctuation=False, remove_special_chars_numbers=False, remove_other_x_chars=False):
  gram_generate = defaultdict(int)

  # Procesando cada opinión
  for index, row in df.iterrows():
      opinion = row['Opinion']
      label = row['Label']
      
      # Preprocesando el texto de la opinión
      tokens = preprocess_text(opinion, remove_stopwords, lemmatize, include_punctuation, remove_special_chars_numbers, remove_other_x_chars)
        
      # Generando trigramas
      n_grams = ngrams(tokens, n)
        
      # Actualizando el puntaje de los trigramas
      for gram in n_grams:
        gram_generate[gram] += 1 if label == 1 else -1
  
  return gram_generate

In [10]:
# Diccionario para almacenar el puntaje de los trigramas
gram_scores = defaultdict(int)

In [11]:
gram_scores = generate_gram(2, True, False, False, True, False)
# Mostrando los trigramas y sus puntajes
for gram, score in gram_scores.items():
  if score > 1 or score < -1:
      print(f'Grama: {gram}, Puntaje: {score}')

Grama: ('momias', 'guanajuato'), Puntaje: 13
Grama: ('museo', 'si'), Puntaje: 4
Grama: ('hace', 'anos'), Puntaje: 9
Grama: ('momias', 'habia'), Puntaje: -2
Grama: ('ambos', 'lados'), Puntaje: -2
Grama: ('bella', 'ciudad'), Puntaje: 15
Grama: ('recomiendo', 'visitarlo'), Puntaje: 8
Grama: ('visitarlo', 'si'), Puntaje: 6
Grama: ('si', 'solo'), Puntaje: 8
Grama: ('solo', 'si'), Puntaje: -3
Grama: ('tiempo', 'pues'), Puntaje: 2
Grama: ('puedes', 'encontrar'), Puntaje: 27
Grama: ('mejor', 'ir'), Puntaje: 7
Grama: ('ir', 'lugar'), Puntaje: 6
Grama: ('guanajuato', 'pues'), Puntaje: 3
Grama: ('asi', 'si'), Puntaje: 4
Grama: ('si', 'van'), Puntaje: 13
Grama: ('van', 'guanajuato'), Puntaje: 9
Grama: ('guanajuato', 'solo'), Puntaje: 4
Grama: ('guanajuato', 'embargo'), Puntaje: 2
Grama: ('embargo', 'esperaba'), Puntaje: -2
Grama: ('lugar', 'interesante'), Puntaje: 6
Grama: ('interesante', 'cada'), Puntaje: -2
Grama: ('cada', 'vez'), Puntaje: 2
Grama: ('escuchado', 'hablar'), Puntaje: -2
Grama: ('g

### Generando Gramas con valores con porcentaje, obteniendo la sumatoria netre positivos y negativos

In [12]:
def generate_gram(n, remove_stopwords=False, lemmatize=False,include_punctuation=False, remove_special_chars_numbers=False, remove_other_x_chars=False):
  gram_generate = defaultdict(int)

  # Contar la cantidad de registros positivos y negativos
  total_positivos = df[df['Label'] == 1].shape[0]
  total_negativos = df[df['Label'] == 0].shape[0]

  # Diccionarios para almacenar la frecuencia de trigramas en opiniones positivas y negativas
  gram_counts_positivos = defaultdict(int)
  gram_counts_negativos = defaultdict(int)

  # Procesando cada opinión
  for index, row in df.iterrows():
      opinion = row['Opinion']
      label = row['Label']
      
      # Preprocesando el texto de la opinión
      tokens = preprocess_text(opinion, remove_stopwords, lemmatize, include_punctuation, remove_special_chars_numbers, remove_other_x_chars)
        
      # Generando trigramas
      n_grams = ngrams(tokens, n)
        
      # Actualizar el conteo de trigramas en opiniones positivas y negativas
      for gram in n_grams:
        if label == 1:
            gram_counts_positivos[gram] += 1
        elif label == 0:
            gram_counts_negativos[gram] += 1

  # Calcular la proporción de cada trigrama en opiniones positivas y negativas
  gram_proportions_positivos = {gram: count / total_positivos for gram, count in gram_counts_positivos.items()}
  gram_proportions_negativos = {gram: count / total_negativos for gram, count in gram_counts_negativos.items()}

  # Unir resultados para mostrar
  all_grams = set(gram_proportions_positivos.keys()).union(set(gram_proportions_negativos.keys()))
  results = {}
  
  for gram in all_grams:
    prop_positivos = gram_proportions_positivos.get(gram, 0)
    prop_negativos = gram_proportions_negativos.get(gram, 0)
    #results[gram] = (prop_positivos, prop_negativos)
    diff = prop_positivos - prop_negativos  # Calcular la diferencia
    results[gram] = diff  # Guardar la diferencia en el diccionario

  return results   

In [13]:
result_frecuency = generate_gram(2, True, False, False, True, False)

# Mostrar resultados
for gram, diff in result_frecuency.items():
    if diff != 0:
        print(f'Gram: {gram}, Diferencia: {diff:.4f}')

Gram: ('danza', 'comedia'), Diferencia: 0.0003
Gram: ('arquitectonico', 'trata'), Diferencia: -0.0013
Gram: ('teatro', 'abierto'), Diferencia: 0.0003
Gram: ('fin', 'artesanias'), Diferencia: 0.0003
Gram: ('partes', 'interesante'), Diferencia: 0.0003
Gram: ('pintoresco', 'duda'), Diferencia: 0.0003
Gram: ('momento', 'llegada'), Diferencia: 0.0003
Gram: ('rededor', 'ojala'), Diferencia: -0.0013
Gram: ('decirles', 'queria'), Diferencia: -0.0013
Gram: ('lleno', 'lugar'), Diferencia: 0.0005
Gram: ('magicos', 'quiero'), Diferencia: 0.0003
Gram: ('observaras', 'callejon'), Diferencia: 0.0003
Gram: ('cansado', 'si'), Diferencia: 0.0003
Gram: ('leyenda', 'tomarse'), Diferencia: 0.0003
Gram: ('proyecciones', 'eventos'), Diferencia: 0.0003
Gram: ('mirador', 'gran'), Diferencia: 0.0003
Gram: ('centro', 'aproximadamente'), Diferencia: 0.0003
Gram: ('ciudad', 'solo'), Diferencia: -0.0005
Gram: ('encontrar', 'dos'), Diferencia: 0.0003
Gram: ('escondido', 'mas'), Diferencia: 0.0003
Gram: ('juarez', 'c